# 0. Install and Import Dependencies

In [ ]:
#run conda install for mediapipe opencv-python pandas scikit-learn numpy

In [ ]:
import mediapipe as mp # Import mediapipe
import cv2 # Import opencv

In [ ]:
mp_drawing = mp.solutions.drawing_utils # Drawing helpers
mp_holistic = mp.solutions.holistic # Mediapipe Solutions

# 1. Detections

In [ ]:
cap = cv2.VideoCapture(0)

# Initialize a resizable window.
cv2.namedWindow('Raw Webcam Feed', cv2.WINDOW_NORMAL)

# Initiate holistic model to start first detection. Then close and run the other sections of the code.

#detection and tracking confidence is kept at default, 0.5
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()

        # Get the width and height of the frame
        frame_height, frame_width, _ =  frame.shape
    
        # Resize the frame while keeping the aspect ratio.
        frame = cv2.resize(frame, (int(frame_width * (640 / frame_height)), 640))

        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        #Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )                        


        cv2.imshow('Raw Webcam Feed', image)
        #exit out of video by pressing q
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# 2. Capture Landmarks & Export to CSV
<!--<img src="https://i.imgur.com/8bForKY.png">-->
<!--<img src="https://i.imgur.com/AzKNp7A.png">-->

In [ ]:
import csv
import os
import numpy as np

In [ ]:
#number of coordsinates = 33 landmarks this is from mediapipe blaze model
num_coords = len(results.pose_landmarks.landmark)  
num_coords

In [ ]:
#format the csv file to include x, y, z, v, data for each landmark.
# class name will be the label to each pose. standing, kneeling, prone
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

2.1 Create a CSV

In [ ]:
#Creates the csv file that holds landmark coordinates
with open('final_training8.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

2.2 Make a new class name

In [ ]:
#Name of poses to be used for training
#class_name = "Shooting Knee"
#class_name = "Shooting stand"
#class_name = "Shooting prone"
#class_name = "Shooting sitting"
#class_name = "Go Gators!"
class_name = "None"




2.3 Export landmark coordinates into csv by live webcam feed or pretrained datasets

In [ ]:
#Run this portion of code to append into the csv the landmark coordiantes of chosen class_name.
#When done with performing the pose stop this cell. 
#Then go back to the class cell and change the class name if you want to perform a diffrent stance 
#Then run this cell again to append to the csv

#If using a pre-recorded video dataset to train from use this line of code 
#cap = cv2.VideoCapture('dataset/shooting_knee.mp4')


#Using webcam to train live
cap = cv2.VideoCapture(0)

#Using pr-recorded datasets to train
#cap = cv2.VideoCapture('dataset/shooting_stand.mp4')

# Initiate holistic model from MediaPipe
with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.7) as holistic:
#with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

    
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
      
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        
        #Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

            # Append class name 
            pose_row.insert(0, class_name)
            
            # Export to CSV
            #The chosen class name will append to the csv wiht all the landmark coordinates
            with open('final_training8.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(pose_row) 
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# 3. Train Custom Model Using Scikit Learn

## 3.1 Read in Collected Data and Process

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
#Run this cell to with name of csv file you want to read into
#csv was our best working file that we used
#df = pd.read_csv('coords.csv')
#df = pd.read_csv('Shooting_Stance.csv')
df = pd.read_csv('final_training7.csv')
#df = pd.read_csv('final_training8.csv')
#df = pd.read_csv('final_training.csv')


In [ ]:
X = df.drop('class', axis=1) # features
y = df['class'] # target value

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

## 3.2 Train Machine Learning Classification Model

In [ ]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [ ]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [ ]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

## 3.3 Evaluate and Serialize Model 

In [ ]:
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 

In [ ]:
#Prints out the 4 ML algorithims with thier accuracy 
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

In [ ]:
#choose the ML algorithm with best accurary. rf(random Forest works the best usually)
with open('body_language.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# 4. Make Detections with Model

In [ ]:
with open('body_language.pkl', 'rb') as f:
    model = pickle.load(f)

In [ ]:
#Run this cell to detect the skeleton
# the classification will now be added ontop of the detection

cap = cv2.VideoCapture(0)

# Initialize a resizable window.
cv2.namedWindow('Raw Webcam Feed', cv2.WINDOW_NORMAL)

# Initiate holistic model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    while cap.isOpened():
        ret, frame = cap.read()

        # Get the width and height of the frame
        frame_height, frame_width, _ =  frame.shape
    
        # Resize the frame while keeping the aspect ratio.
        frame = cv2.resize(frame, (int(frame_width * (640 / frame_height)), 640))
        
        # Recolor Feed
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False        
        
        # Make Detections
        results = holistic.process(image)
        
        # Recolor image back to BGR for rendering
        image.flags.writeable = True   
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
 

        #Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        # Export coordinates
        try:
            # Extract Pose landmarks
            pose = results.pose_landmarks.landmark
            pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
         
            # Make Detections
            #X = pd.DataFrame([row])
            X = pd.DataFrame([pose_row])
            body_language_class = model.predict(X)[0]
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            # Grab ear coordinates
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x, 
                                    results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y))
                        , [640,480]).astype(int))
            
            cv2.rectangle(image, 
                            (coords[0], coords[1]+5), #x, y cordinates
                            (coords[0]+len(body_language_class)*20, coords[1]-30), 
                            (0, 140,255), -1) #BGR color of rectangle and filled = -1
            cv2.putText(image, body_language_class, coords, 
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            
            # Get status box
            #cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            #topleft corner, rectangle length and width, color, fill
            cv2.rectangle(image, (0,0), (350, 60), (0, 140, 255), -1)
            
            # Display Class
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            #cv2.putText(image, body_language_class.split(' ')[0]
            cv2.putText(image, body_language_class
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            
            # Display Probability
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                        , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
            
        except:
            pass
                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()